In [31]:
import doekbase.data_api
from doekbase.data_api.annotation.genome_annotation.api import GenomeAnnotationAPI
import time
import os

In [46]:
def get_gff(gene,genome_annotation, taxon_id, source_info, count, contig_len):
    
    print(gene)
    #retrieve pairwise inter-feature relationships between genes, mRNAs, CDSs
    #the below section of code has method timings torecord performance  
    t0 = time.time()
    listmrna=genome_annotation.get_mrna_by_gene([gene])[gene]    
    t1 = time.time()
    print "get_mrna_by_gene " , (t1-t0), "s"
    #print(listmrna)
    
    #t0 = time.time()
    #listcds=genome_annotation.get_cds_by_gene([gene])[gene] 
    #t1 = time.time()
    #print "get_cds_by_gene " , (t1-t0), "s"   
    #print(listcds)
        
    if len(listmrna) > 0:
        t0 = time.time()
        print(listmrna)
        mrna_cds=genome_annotation.get_cds_by_mrna(listmrna)
        t1 = time.time()
        print "get_cds_by_mrna " , (t1-t0), "s"
        #print(mrna_cds)
    
        #t0 = time.time()
        #cds_mrna=genome_annotation.get_mrna_by_cds(listcds)
        #t1 = time.time()
        #print "get_mrna_by_cds " , (t1-t0), "s"

        t0 = time.time()
        get_mrna_exons = genome_annotation.get_mrna_exons(listmrna)
        t1 = time.time()
        print "get_mrna_exons " , (t1-t0), "s"
        #print get_mrna_exons
        
        #retrieve locations for mRNAs
        t0 = time.time()
        mrna_location=genome_annotation.get_feature_locations(listmrna) 
        t1 = time.time()
        print "get_feature_locations " , (t1-t0), "s"


    #retrieve locations for genes, mRNAs, and cds
    t0 = time.time()
    gene_location=genome_annotation.get_feature_locations([gene]) 
    t1 = time.time()
    print "get_feature_locations " , (t1-t0), "s"
    
    #t0 = time.time()
    #cds_location=genome_annotation.get_feature_locations(listcds) 
    #t1 = time.time()
    #print "get_feature_locations " , (t1-t0), "s"
    
    string=""
    
    if count == 0:
        string+="##sequence-region "+gene_location[gene_location.keys()[0]][0]['contig_id']+"\t"+"1"+"\t"+str(contig_len)+"\n"##NC_003070.9 1 30427671
        if taxon_id != -1:
            string+="##species http://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id="+str(taxon_id)+"\n"#http://www.ncbi.nlm.nih.gov/Taxonomy/Browser/wwwtax.cgi?id=3702
        else :
            string+="##species unknown\n"
    

    contig_id = ""
    #output the gene data
    for x in gene_location:
        #print(type(x))
        #print(x)
        info=gene_location[x][0]
        #print(type(info))
        #print info
        stop=info['start'] + info['length']
        
        contig_id = info['contig_id']
        
        #print "source_info"
        #print(type(source_info))
        #print(source_info)
        #output aliases for gene as dbxref?
        string+=info['contig_id']  + "\t" + source_info['external_source']+"\tgene\t"  + str(info['start']) + "\t" + str(stop) + "\t.\t" +  info['strand']+ "\t.\t" + "ID=" + x + ";" + "Name="+ x +"\n"
    
    print "len(listmrna)"    
    print(len(listmrna))
    if len(listmrna) > 0:    
    #output the mRNA data
        for mrna in listmrna:
            print("mrna "+mrna)
            cds=mrna_cds[mrna]
            infomrna=mrna_location[mrna][0]
            #print(type(infomrna))
            #print(infomrna)
            #infocds=cds_location[mrna_cds[mrna]]
            stop=infomrna['start'] + infomrna['length']            
            ###output mrna data
            string+=infomrna['contig_id']  + "\t" +source_info['external_source']+"\tmRNA\t"  + str(infomrna['start'])+ "\t" + str(stop) + "\t.\t" +  infomrna['strand']+ "\t.\t" + "ID=" + mrna + ";" + "Parent="+ gene+"\n"
            
            #print 'get_mrna_exons.keys()'
            #print get_mrna_exons.keys()
            
            ##TODO output UTRs
            for mrna in get_mrna_exons.keys():
                #print(type(get_mrna_exons[mrna]))
                #print(get_mrna_exons[mrna])
                #print "get_mrna_exons[mrna][i]['exon_location']"
                #print type(get_mrna_exons[mrna][0]['exon_location'])
                ###output exon data
                for i in range(0, len(get_mrna_exons[mrna])):
                    #print "mrna, gene"
                    #print type(mrna)
                    #print type(gene)
                    stop= get_mrna_exons[mrna][i]['exon_location']['start'] + get_mrna_exons[mrna][i]['exon_location']['length']
                    string+= get_mrna_exons[mrna][i]['exon_location']['contig_id']  + "\t" +source_info['external_source'] + "\texon\t"  + str(get_mrna_exons[mrna][i]['exon_location']['start'])+ "\t" + str(stop) + "\t.\t" +  get_mrna_exons[mrna][i]['exon_location']['strand']+ "\t.\t" + "ID=" + mrna +".exon." + str(i)  + ";" + "Parent="+ mrna+"\n"
                ###output CDS data
                for i in range(0, len(get_mrna_exons[mrna])):
                    stop= get_mrna_exons[mrna][i]['exon_location']['start'] + get_mrna_exons[mrna][i]['exon_location']['length']
                    ##***TODO: set PHASE for CDS
                    string+= get_mrna_exons[mrna][i]['exon_location']['contig_id']  + "\t" +source_info['external_source'] + "\tCDS\t"  + str(get_mrna_exons[mrna][i]['exon_location']['start'])+ "\t" + str(stop) + "\t.\t" +  get_mrna_exons[mrna][i]['exon_location']['strand']+ "\t.\t" + "ID=" + mrna +".CDS." + str(i)  + ";" + "Parent="+ mrna+"\n"

    return string       

#hardcoded test with 5 random genes
def runDemo(ws_url='https://ci.kbase.us/services/ws/', ws="ReferenceEnsemblPlantGenomeAnnotations", genome_id="280699_Ensembl"):
    print "1"
    #an example KBase reference genome
    genome_ref = "ReferenceEnsemblPlantGenomeAnnotations"+"/"+genome_id
    #instantiate a new genome annotation API
    t0 = time.time()
    genome_annotation = GenomeAnnotationAPI(services = {"workspace_service_url": ws_url}, token=os.environ.get('KB_AUTH_TOKEN'), ref=genome_ref)
    t1 = time.time()
    print "GenomeAnnotationAPI " , (t1-t0), "s"
    
    t0 = time.time()
    taxon = genome_annotation.get_taxon()
    t1 = time.time()
    print "get_taxon " , (t1-t0), "s"
        
    t0 = time.time()
    taxonid = taxon.get_taxonomic_id()
    t1 = time.time()
    print "get_taxonomic_id " , (t1-t0), "s"
    
    if taxonid == -1 :
        taxonid = "unknown"
    
    assembly = genome_annotation.get_assembly()
    contig_ids = assembly.get_contig_ids()
    sourceinfo = assembly.get_external_source_info()
    genes=['gene_1','gene_2','gene_5', 'gene_100', 'gene_1000']
    #genes=['gene_1']
    
    gffdata=""
    gffdata+="##gff-version 3\n"
    gffdata+="#!gff-spec-version 1.21\n"
    gffdata+="#!processor KBase GFF3 downloader\n"
    #!genome-build TAIR10
    #!genome-build-accession NCBI_Assembly:GCF_000001735.3   
    
    count = 0
    for s in genes:
        gffdata+=get_gff(s,genome_annotation,taxonid, sourceinfo, count, -1)
        count = count + 1

    gffdata+="###"
    outfile = genome_id+'.gff'
    print outfile
    with open(outfile, 'w') as f:
        f.write(gffdata)
    
#create GFF for an entire GenomeAnnotation
def runGenome(ws_url='https://ci.kbase.us/services/ws/', ws="ReferenceEnsemblPlantGenomeAnnotations", genome_id="280699_Ensembl"):
    #an example KBase reference genome
    genome_ref = "ReferenceEnsemblPlantGenomeAnnotations"+"/"+genome_id#kb|g.166819"
    #instantiate a new genome annotation API
    t0 = time.time()
    genome_annotation = GenomeAnnotationAPI(services = {"workspace_service_url": ws_url}, token=os.environ.get('KB_AUTH_TOKEN'), ref=genome_ref)
    t1 = time.time()
    print "GenomeAnnotationAPI " , (t1-t0), "s"
    
    t0 = time.time()
    taxon = genome_annotation.get_taxon()
    t1 = time.time()
    print "get_taxon " , (t1-t0), "s"
        
    t0 = time.time()
    taxonid = taxon.get_taxonomic_id()
    t1 = time.time()
    print "get_taxonomic_id " , (t1-t0), "s"
    
    print "1"
    
    if taxonid == -1 :
        taxonid = "unknown"
    
    assembly = genome_annotation.get_assembly()
    contig_ids = assembly.get_contig_ids()
    sourceinfo = assembly.get_external_source_info()
    
    contig_lens = assembly.get_contig_lengths()
    #print contig_lens
    
    print "2"
    
    gffdata=""
    gffdata+="##gff-version 3\n"
    gffdata+="#!gff-spec-version 1.21\n"
    gffdata+="#!processor KBase GFF3 downloader\n"
    #!genome-build TAIR10
    #!genome-build-accession NCBI_Assembly:GCF_000001735.3  
    
    start=1
    stop=100000000
    
    feat_data = []
    print "contig_ids len "+ str(len(contig_ids))
    countc = 0
    feat_total = 0
    for c in contig_ids:
        
        stop = contig_lens[c]
        print c
        regions = []
        contig_add = {"contig_id": c, "start": start, "length": stop-start, "strand": "+"}
        contig_add2 = {"contig_id": c, "start": stop, "length": stop-start, "strand": "-"}
        regions.append(contig_add)
        regions.append(contig_add2)
        
        t0 = time.time()
        feat_data = genome_annotation.get_feature_ids(filters={"region_list":regions,"type_list":["gene"]})['by_type']['gene']
        t1 = time.time()
        print "get_feature_ids " , (t1-t0), "s"
    
        print "feat_data len "+ str(len(feat_data))
        #print feat_data
        #for s in genes:
        countf =0
        feat_total = feat_total + len(feat_data)
        for f in feat_data:
            print f
            gffdata+=get_gff(f,genome_annotation,taxonid, sourceinfo, countf, stop)
            #print gffdata
            countf = countf + 1
            #break after small number       
            if countf > 2:
                print "break after first 3 genes for now"
                break
               
        countc = countc + 1
        if countc > 2:
            print "break after first 3 contigs for now "   
            #break after first contig for now    
            break
    
    #print "END"
    #print gffdata
    
    print "process contigs " + str(countc)
    print "feat total " + str(feat_total)
    
    gffdata+="###"
    outfile = genome_id+'.gff'
    print outfile
    with open(outfile, 'w') as f:
        f.write(gffdata)
        
    feat_data

In [12]:
genomeid="280699_Ensembl"
wsthis = "ReferenceEnsemblPlantGenomeAnnotations"
###Note that this demo function does not produce biologically valid GFF
runDemo(ws=wsthis, genome_id=genomeid)

1
GenomeAnnotationAPI  4.83470010757 s
get_taxon  3.01157689095 s
get_taxonomic_id  2.71797180176e-05 s
gene_1
get_mrna_by_gene  2.03141212463 s
[u'mRNA_1']
get_cds_by_mrna  2.27638292313 s
get_mrna_exons  2.40531301498 s
get_feature_locations  3.17668700218 s
get_feature_locations  3.62060785294 s
len(listmrna)
1
mrna mRNA_1
gene_2
get_mrna_by_gene  2.80000805855 s
[u'mRNA_2']
get_cds_by_mrna  2.54419612885 s
get_mrna_exons  3.1331140995 s
get_feature_locations  3.34578895569 s
get_feature_locations  3.85967493057 s
len(listmrna)
1
mrna mRNA_2
gene_5
get_mrna_by_gene  3.37133693695 s
[u'mRNA_3']
get_cds_by_mrna  2.73723483086 s
get_mrna_exons  3.23955893517 s
get_feature_locations  3.27452707291 s
get_feature_locations  5.13207197189 s
len(listmrna)
1
mrna mRNA_3
gene_100
get_mrna_by_gene  4.27790999413 s
[u'mRNA_63']
get_cds_by_mrna  3.72632789612 s
get_mrna_exons  3.1912920475 s
get_feature_locations  3.42246103287 s
get_feature_locations  2.94254207611 s
len(listmrna)
1
mrna mRNA_6

In [47]:
genomeid="280699_Ensembl"
wsthis = "ReferenceEnsemblPlantGenomeAnnotations"
runGenome(ws=wsthis, genome_id=genomeid)

ConnectionError: HTTPSConnectionPool(host='ci.kbase.us', port=443): Max retries exceeded with url: /services/ws/ (Caused by <class 'socket.gaierror'>: [Errno 8] nodename nor servname provided, or not known)

In [ ]:
#write overall header
#for contigs
    #get contig features
    #write contig header
    #for gene in contig
        #write GFF gene
            #for mRNA from gene
                #write GFF mRNA for gene
                    #for exon in mRNA
                        #write GFF exon for mRNA
                        #write GFF CDS for mRNA
